<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
def tokenize(doc):
    
    tokens = re.sub(r'[^a-zA-Z ^0-9]', '', doc)
    tokens = tokens.lower().split()
    
    return tokens

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [4]:
import re
import spacy
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
def clean_soup(df_column, spec_chars_remove = []):
    """
    Input: dataframe column and list of specific characters to remove, 
    
    Output: List of cleaned observations
    """
    
    soupy = [BeautifulSoup(df_column[ii], 'lxml').get_text() 
             for ii in range(df_column.shape[0])
            ]
    
    for char in spec_chars_remove:
        soupy = [soupy[ii].replace(char, ' ') 
                 for ii in range(len(soupy))
                ]
        
    to_clean = ['[^A-Za-z ]+', '   ', '  ']
    
    for char in to_clean:
        soupy = [re.sub(char, ' ', soupy[ii]) 
                 for ii in range(len(soupy))
                ]
        
    df_feature = pd.Series([nlp(soupy[ii].lower().strip()) 
                            for ii in range(len(soupy))
                           ])
        
    for row in range(df_feature.shape[0]):
        df_feature[row] = " ".join([token.lemma_ 
                                    for token in df_feature[row]
                                    if len(str(token)) > 2
                                   ])
         
    return df_feature

In [7]:
yelp['text_cleaned'] = clean_soup(yelp['text'])

In [8]:
yelp['text_cleaned'].head()

0    beware fake fake fake also own small business ...
1    come here for lunch togo service be quick staf...
2    be vegas dozen time and have never step foot i...
3    go here night where -PRON- close off part the ...
4    star not bad for the price for lunch senior ge...
Name: text_cleaned, dtype: object

In [9]:
#vectorize cleaned text
stop_words = nlp.Defaults.stop_words.union(['-PRON-', 'review', 'pron', 'star', 'll','ve'])

tfidf = TfidfVectorizer(stop_words = stop_words, max_features = 8000)

dtm = tfidf.fit_transform(yelp['text_cleaned'])

dtm = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())

dtm.head()

,aaa,abandon,abc,ability,able,abordable,abruptly,absinthe,absolute,absolutely,...,zip,ziti,zombie,zone,zoo,zoom,zroif,zucchini,zumba,zuzana
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [11]:
fake_review = ["""This was the greatest place to have lunch, 
                    the atmosphere was great, food was great, 
                    service was outstanding. Cheap prices, 
                    great lunch deals"""]

fake = tfidf.transform(fake_review)

In [12]:
nearest_vectors, nearest_loc = nn.kneighbors(fake.todense())

for neighbor in list(nearest_loc):
    print(f"""Stars:\n {yelp['stars'].iloc[neighbor]}\n
                Review:\n {yelp['text'].iloc[neighbor][0:300]}
                Review Cleaned:\n {yelp['text_cleaned'].iloc[neighbor][0:300]}""")

Stars:
 6625    3
6204    5
469     4
6311    5
1192    5
7594    5
2199    3
7486    4
1522    5
555     3
Name: stars, dtype: int64

                Review:
 6625    Food and service are ok. There are much better...
6204    旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。\n質問にも丁寧...
469     O  o  thenk 6nnn  .b  cgv  xx TV cvg  9 nvehxc...
6311    天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用...
1192    This place is so cute! It has a great atmosphe...
7594    Great place to hang out SteveO is awesome and ...
2199    Came for lunch and didn't see much of a crowd ...
7486    Their view is amazing. Go there for LUNCH TIME...
1522    This is a great Asian food place. Great sushi,...
555     Chingu Korean BBQ's Lunch Specials for $5.99! ...
Name: text, dtype: object
                Review Cleaned:
 6625    food and service be there be much well place f...
6204                                                     
469     thenk nnn cgv cvg nvehxcfvvv and the vghvhridd...
6311                       

The reviews found by nearest neighbors model are all rated with 3 and higher stars, 
and they mostly use similar language to which I used in the fake review

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from scipy.stats import randint, uniform

In [14]:
X_train, X_val, y_train, y_val = train_test_split(yelp['text_cleaned'], yelp['stars'],
                                                  test_size=0.20, random_state=42, stratify = yelp['stars'])

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((8000,), (2000,), (8000,), (2000,))

In [15]:
vect = TfidfVectorizer(stop_words=stop_words)

rfc = RandomForestClassifier()

svd = TruncatedSVD(algorithm='randomized')

In [18]:
# pipline using latent semantic indexing

lsi = Pipeline([('vect', vect), 
                ('svd', svd)])

pipeline = Pipeline([('lsi', lsi),
                 ('clf', rfc)])

# The pipeline puts together a bunch fit then transform,fit then predict.
parameters = {
    'lsi__vect__ngram_range': [(1,2)],
    'lsi__vect__max_df': [.85, .95],
    'lsi__vect__min_df': [.01, .1],
    'lsi__vect__max_features': [500, 5000],
    'lsi__svd__n_components': [30],
    'clf__n_estimators': [ 100, 500],
    'clf__class_weight': ['balanced'] 
}

search = GridSearchCV(
    pipeline, 
    parameters,  
    cv=3, 
    verbose = 10,
    n_jobs = -1
)
search.fit(X_train, y_train)

print(f'Best score: {search.best_score_}\n')
print(f'Best hyperparameters: \n{search.best_params_}\n')

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  42 out of  48 | elapsed:  2.9min remaining:   25.1s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  3.4min finished


Best score: 0.554125

Best hyperparameters: 
{'clf__class_weight': 'balanced', 'clf__n_estimators': 100, 'lsi__svd__n_components': 30, 'lsi__vect__max_df': 0.85, 'lsi__vect__max_features': 5000, 'lsi__vect__min_df': 0.01, 'lsi__vect__ngram_range': (1, 2)}



In [19]:
best_pipeline = search.best_estimator_
best_pipeline.fit(X_train, y_train)

print(f'Validation Accuracy:\n{best_pipeline.score(X_val, y_val)}')

Validation Accuracy: 
0.5455



In [27]:
fake_review_pred = best_pipeline.predict(fake_review)

print(f"""
The model predicts my fake review would give
the location a rating of {fake_review_pred[0]} stars.
""")


The model predicts my fake review would give
the location a rating of 5 stars.



## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [42]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from spacy.tokenizer import Tokenize
import pyLDAvis.gensim
import warnings #for LDA warnings

Learn the vocubalary of the yelp data:

In [29]:
def make_tokens(df_feature, addl_stop_words = ['-PRON-']):
    """
    Input: Column of a dataframe/ Pandas Series, 
    stop words you'd like to add to nlp's defaults
    
    Output: List consisting of tokens for each observation
    
    Assumes: nlp object initialized as nlp
    """
    
    tokens = []
    tokenizer = Tokenizer(nlp.vocab)

    STOP_WORDS = nlp.Defaults.stop_words.union(addl_stop_words)

    for doc in tokenizer.pipe(df_feature, batch_size=500):

        doc_tokens = []

        for token in doc: 
            if token.text not in STOP_WORDS:
                doc_tokens.append(token.text.lower())

        tokens.append(doc_tokens)

    return tokens

In [30]:
yelp['text_tokens'] = make_tokens(yelp['text_cleaned'], stop_words)
yelp['text_tokens'].head()

0    [beware, fake, fake, fake, small, business, lo...
1    [come, lunch, togo, service, quick, staff, fri...
2    [vegas, dozen, time, step, foot, circus, circu...
3    [night, close, street, party, good, actually, ...
4    [bad, price, lunch, senior, pay, eat, hot, foo...
Name: text_tokens, dtype: object

In [31]:
id2word = Dictionary(yelp['text_tokens'])
id2word.filter_extremes(no_below=5, no_above=0.95)

Create a bag of words representation of the entire corpus

In [32]:
corpus = [id2word.doc2bow(text) for text in yelp['text_tokens']]

Your LDA model should be ready for estimation: 

In [44]:
warnings.filterwarnings("ignore",category=DeprecationWarning)

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=3,
                   workers=6,
                   num_topics = 3 # tuned after initial results of pyLDAvis
                  )

In [45]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

topics = [' '.join(t[0:3]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
place good food

------ Topic 1 ------
good place food

------ Topic 2 ------
good place food



Create 1-2 visualizations of the results

In [46]:
warnings.filterwarnings("ignore",category=FutureWarning)

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.004121  0.001473       1        1  33.394123
2     -0.000624 -0.003904       2        1  33.314503
0     -0.003497  0.002432       3        1  33.291378, topic_info=    Category         Freq        Term        Total  loglift  logprob
329  Default  3451.000000       order  3451.000000  30.0000  30.0000
36   Default  6467.000000        good  6467.000000  29.0000  29.0000
31   Default  2555.000000         try  2555.000000  28.0000  28.0000
41   Default  3801.000000     service  3801.000000  27.0000  27.0000
33   Default  3885.000000        come  3885.000000  26.0000  26.0000
..       ...          ...         ...          ...      ...      ...
323   Topic3   481.404175      little  1391.901489   0.0382  -5.7072
429   Topic3   478.637238        tell  1508.808960  -0.0483  -5.7130
142   Topic3   477.040009       think  1510.241577  -0.0526  -5.7163
231   Topic3   471.824341  experience  1452.904297  -0.0248  -5.7273
280   Topic3   472.303345        work  1526.123657  -0.0730  -5.7263

[279 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1975      1  0.488860    accept
1975      2  0.268085    accept
1975      3  0.236545    accept
105       1  0.434172  actually
105       2  0.268106  actually
...     ...       ...       ...
602       2  0.430429     worth
602       3  0.242304     worth
640       1  0.494699     wrong
640       2  0.267025     wrong
640       3  0.238917     wrong

[597 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)